# 지도 학습 (5) - 결과 보고
- 논문 등의 보고서에서 `결과 보고`를 위해 `회귀계수 표`를 구성하기 위한 값을 구하는 과정
## 1. 작업 준비

In [46]:
import sys
sys.path.append("../../")
import helper

import numpy as np
from scipy import stats
from pandas import read_excel, DataFrame
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [47]:
origin = read_excel('https://data.hossam.kr/F02/fish2.xlsx')
origin.head()

,길이,높이,두께,무게
0,8.4,2.11,1.41,5.9
1,13.7,3.53,2.00,32.0
2,15.0,3.82,2.43,40.0
3,16.2,4.59,2.63,51.5
4,17.4,4.59,2.94,70.0


## 2. 머신러닝에 의한 회귀분석 수행

In [48]:
xnames = ['길이', '높이', '두께']
yname = '무게'

In [49]:
# 머신러닝 전에 사용했던 ols 회귀분석과 비교하기 위하여 
# 2차항으로 변환하지 않을 것, 훈련/검증 데이터로도 분리하지 않을 것
result = helper.ml_ols(origin, xnames, yname, degree = 1, test_size = 0)
print('계수: ', result.coef)
print('절편: ', result.intercept)
result.table

계수:  [ 2.9082713  67.20469902 67.26029602]
절편:  -546.4397914448649


,종속변수,독립변수,B,표준오차,β,t,유의확률,VIF
0,무게,길이,2.908271,7.733688,0,0.376052,0.708408,338.760305
1,무게,높이,67.204699,29.447158,0,2.282213,0.026598,500.757056
2,무게,두께,67.260296,36.011233,0,1.867759,0.067436,263.015058


## 3. 결과보고에 필요한 값 구하기
### 1) 절편과 계수를 하나의 배열로 결합

In [50]:
params = np.append(result.intercept, result.coef)
params

array([-546.43979144,    2.9082713 ,   67.20469902,   67.26029602])

### 2) `상수항 추가하기`
- 각종 api들을 사용할 떄 api가 내부적으로 상수항이 있다라고 처리해주기에 api를 사용할 때에는 상수항을 인지할 필요가 없으나 지금은 우리가 회귀분석에서 사용되는 모든 값들을 직접 계산을 하고 있기에 상수항을 우리가 직접 추가 해줘야 된다.
- 이 `상수항 (1)`에다가 `절편을 곱하는 방식`이다.

In [65]:
# 독립변수 추출
x = origin.filter(xnames)
# 종속변수 추출
y = origin[yname]       # 이 때 y는 시리즈 객체
# 상수항 추가
designX = x.copy()
designX.insert(0, '상수', 1)    # .insert(추가할열위치, '추가할열이름', 넣을값)
designX.head()

,상수,길이,높이,두께
0,1,8.4,2.11,1.41
1,1,13.7,3.53,2.00
2,1,15.0,3.82,2.43
3,1,16.2,4.59,2.63
4,1,17.4,4.59,2.94


### 3) `행렬곱` 구하기
- `numpy.dot(데이터프레임1, 데이터프레임2)`
- 방법: 주어진 데이터프레임의 역행렬에 원래 데이터프레임 곱하기

In [53]:
dot = np.dot(designX.T, designX)
dot

array([[   56.    ,  1562.    ,   440.28  ,   265.75  ],
       [ 1562.    , 48045.12  , 13688.339 ,  8270.876 ],
       [  440.28  , 13688.339 ,  3917.2114,  2365.5425],
       [  265.75  ,  8270.876 ,  2365.5425,  1434.4117]])

### 4) 행렬곱에 대한 `역행렬`
- `numpy.linalg.inv()`

In [54]:
inv = np.linalg.inv(dot)
inv

array([[ 0.25997581, -0.02937614,  0.05587393,  0.02907514],
       [-0.02937614,  0.00811062, -0.0207489 , -0.00710593],
       [ 0.05587393, -0.0207489 ,  0.11758923, -0.08463348],
       [ 0.02907514, -0.00710593, -0.08463348,  0.17585582]])

### 5) 역행렬의 `대각선 값` 추출
- inv.diagonal()
- 위 값에서 [0][0], [1][1], [2][2], [3][3]의 값을 구해준다.

In [9]:
dia = inv.diagonal()
dia

array([0.25997581, 0.00811062, 0.11758923, 0.17585582])

### 6) 평균 제곱오차 구하기
- 상수항이 적용된 형태이므로 API를 통한 값이 아닌 직접 구한 값이 필요하다
- 아까 구한 result라는 결과에 `.fit.predict()` 함수를 적용하면 예측표가 나온다. 

In [37]:
predictions = result.fit.predict(x).reshape(-1)
# MSE = 잔차의 제곱 / ((전체 데이터 개수) - (첫번째 행의 값의 개수))
MSE = (sum((y-predictions)**2)) / (len(designX)-len(designX.iloc[0])) # 평균 제곱 오차 값
MSE

7374.273394715796

### 7) 표준 오차 값들 구하기
- `numpy.sqrt()`는 제곱근을 구하는 함수

In [11]:
se_b = np.sqrt(MSE * dia)
se_b

array([43.78507388,  7.73368804, 29.44715768, 36.0112326 ])

### 8) t-value 구하기
- 절편과 계수들 (array([-546.43979144, 2.9082713, 67.20469902, 67.26029602])을 표준오차 값들로 나눈 값이 t-value들이다.
- t-value은 `표본 데이터 변동에 비례한 차이의 크기를 측정`
- T는 `계산된 차이를 표준 오차 단위로 나타낸 것`
- T의 크기가 `클수록 귀무 가설에 대한 증거가 큽니`다 즉, `유의한 차이가 있다는 증거가 더 명확한 것`
- 반면 T가 0에 가까울수록 유의미한 차이가 없을 가능성이 커짐

In [38]:
ts_b = params / se_b
ts_b

array([-12.48004726,   0.37605232,   2.28221344,   1.86775878])

### 9) p-value 구하기

In [39]:
p_values = [2*(1-stats.t.cdf(np.abs(i), (len(designX)-len(designX.iloc[0])))) for i in ts_b]
p_values

[0.0, 0.7084079152880389, 0.026597717787693265, 0.06743585337091984]

### 10) VIF 구하기
- VIF (Variance Inflation Factor)란, `분산 팽창 인수`라고 합니다
    - 이 값은 다중회귀분석에서 독립변수가 `다중 공산성(Multicollnearity)`의 `문제를 갖고 있는지 판단`하는 기준이며, 주로 `10보다 크면 그 독립변수는 다중공산성이 있다`고 말합니다.

In [14]:
vif = []
for i, v in enumerate(xnames):
    j = list(origin.columns).index(v)
    vif.append(variance_inflation_factor(origin, j))
vif

[338.76030542544714, 500.757055790855, 263.01505845905143]

### 11) 결과표 구성하기

In [43]:
resultDf = DataFrame({
    "종속변수": [yname] * len(xnames),    # -> 출력: ['무게', '무게', '무게']
    "독립변수": xnames,
    "B": result.coef.reshape(-1),       # 각 피처의 계수
    "표준오차": se_b[1:].reshape(-1),     # 상수항 제외 -> [1:]
    "β": 0,                             # 나중에 배울 것임
    "t": ts_b[1:].reshape(-1),          # 상수항 제외 -> [1:]
    "유의확률": p_values[1:],             # 상수항 제외 -> [1:]
    "VIF": vif                          # 다중공선성 판단 여부
})
resultDf

,종속변수,독립변수,B,표준오차,β,t,유의확률,VIF
0,무게,길이,2.908271,7.733688,0,0.376052,0.708408,338.760305
1,무게,높이,67.204699,29.447158,0,2.282213,0.026598,500.757056
2,무게,두께,67.260296,36.011233,0,1.867759,0.067436,263.015058


### 12) statsmodels 패키지의 결과와 비교해보기
- -> 기존에 사용하던 statsmodels의 회귀분석과 결과가 똑같은지 비교해보는 것

In [44]:
result = helper.myOls(origin, x=['길이', '높이', '두께'], y = '무게')
result.table

B    표준오차  β       t   유의확률         VIF
종속변수 독립변수                                               
무게   길이     2.9083   7.734  0  0.376*  0.708  338.760305
     높이    67.2047  29.447  0  2.282*  0.027  500.757056
     두께    67.2603  36.011  0  1.868*  0.067  263.015058

> 계수의 값(B)이 반올림 된 것을 고려하면 `머신러닝으로 얻어진 분석 결과값`이 `statsmodels(회귀분석 모듈)가 주는 분석 결과값`과 거의 동일하다는 것을 확인
> 
> 그러므로 statsmodels을 사용하던, scikit-learn을 사용하던 둘 다 값이 거의 동일하여 상관 없을 것이다.
### 13) 모듈에 추가된 기능 확인하기

In [71]:
ols_result = helper.ml_ols(origin, xnames = ['길이', '높이', '두께'], yname = '무게',
                           degree = 1, test_size = 0)
ols_result.table

,종속변수,독립변수,B,표준오차,β,t,유의확률,VIF
0,무게,길이,2.908271,7.733688,0,0.376052,0.708408,338.760305
1,무게,높이,67.204699,29.447158,0,2.282213,0.026598,500.757056
2,무게,두께,67.260296,36.011233,0,1.867759,0.067436,263.015058


## 참고자료 (reference)
- https://m.blog.naver.com/pmw9440/222482746927
- https://stackoverflow.com/questions/27928275/find-p-value-significance-in-scikit-learn-linearregression
- https://stats.stackexchange.com/questions/85943/how-to-derive-the-standard-error-of-linear-regression-coefficient
- https://calcworkshop.com/linear-regression/t-test/
- https://www.google.com/url?sa=i&url=http%3A%2F%2Fwww.few.vu.nl%2F~wvanwie%2FCourses%2FHighdimensionalDataAnalysis%2FWNvanWieringen_HDDA_Lecture234_RidgeRegression_20182019.pdf&psig=AOvVaw31slQKGfkCNf2PAa3VEIGC&ust=1630070470635000&source=images&cd=vfe&ved=0CAsQjRxqFwoTCIjJworkzvICFQAAAAAdAAAAABAR